In [45]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (DataLoader,)  # Gives easier dataset managment and creates mini batches
#import torchvision  # torch package for vision related things
#import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
#import torchvision.transforms as transforms
import pandas as pd
import random
from tqdm.auto import tqdm
from transformers import DistilBertTokenizer

In [46]:
# Run when executing on Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

#!cp "/content/drive/MyDrive/Colab Notebooks/Deep Learning/DeepLProject/SpamDataset.zip" "./SpamDataset.zip"
#!unzip SpamDataset.zip

In [58]:
# Split the downloaded Youtube01-Psy.csv file into train, validation and test sets
data = pd.read_csv('SpamDataset/Youtube01-Psy.csv')
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data
train_data = data[:int(0.7*len(data))]
val_data = data[int(0.7*len(data)):int(0.85*len(data))]
test_data = data[int(0.85*len(data)):]

# Tokenize the data using distillbert
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Turn pd dataframe into a tokenized PyTorch dataset that DataLoader can use via the SpamDataset class
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        comment_tokens = row['CONTENT']
        label = row['CLASS']
        return comment_tokens, label
    
    def collate_fn(self, batch):
        comments = [row[0] for row in batch]
        labels = [row[1] for row in batch]
        tokens = tokenizer.batch_encode_plus(comments, pad_to_max_length=True, return_tensors='pt')
        return tokens, torch.tensor(labels)
    
train_dataset = SpamDataset(train_data)
val_dataset = SpamDataset(val_data)
test_dataset = SpamDataset(test_data)


# Load Data and collate it
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=val_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate_fn)

In [70]:
# Print a random sample from the train_loader to confirm correct loading and tokenization
sample = next(iter(train_loader))
print(sample[0]['input_ids'][0])
print(sample[0]['attention_mask'][0])
print(sample[1][0])
print([tokenizer.decode(sample[0]['input_ids'][0][i]) for i in range(len(sample[0]['input_ids'][0]))])

# Print the size of the sample
max_input_size = len(sample[0]['input_ids'][0])

tensor([  101,  7858,  1013, 13326,  3406, 20434,  2239,  2005, 12476,  6876,
         1045,  1005,  2222,  4942, 29234,  2067,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

c:\Users\a_sta\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [71]:
# Recurrent neural network with LSTM (many-to-one)
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, sequence_length):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)  
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)     

    def forward(self, x, device):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out) 
        return out

In [73]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = max_input_size    # Grab Max input size from the sample above
hidden_size = 256
num_layers = 3
num_classes = 2
sequence_length = 28
learning_rate = 0.005
batch_size = 64
num_epochs = 3

# Load Data
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# init model
model_LSTM = RNN_LSTM(input_size, hidden_size, num_layers, num_classes, sequence_length).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_LSTM = optim.Adam(model_LSTM.parameters(), lr=learning_rate)

# RNN_LSTM
print('RNN_LSTM Training')
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores_LSTM = model_LSTM(data)
        loss_LSTM = criterion(scores_LSTM, targets)

        # backward
        optimizer_LSTM.zero_grad()
        loss_LSTM.backward()

        # gradient descent update step/adam step
        optimizer_LSTM.step()

RNN_LSTM Training


  0%|          | 0/4 [00:00<?, ?it/s]

AttributeError: 'tuple' object has no attribute 'to'